In [1]:
import tensorflow 
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
from matplotlib.image import imread

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import sys
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from IPython.display import display
from PIL import Image

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import random 
from tensorflow.keras import models
from tensorflow.keras import backend as K

import pandas as pd

from imgaug import augmenters as iaa
import imgaug as ia
import imageio

In [2]:
DATA_PATH = 'C:\\Users\\pacol\\Desktop\\GIT\\COVID19\\data\\'
NOR_DIR = DATA_PATH + 'New Data\\NORMAL'
PNE_DIR = DATA_PATH + 'New Data\\PNEUMONIA'
COV_DIR = DATA_PATH + 'New Data\\COVID-19'

IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128

In [3]:
def create_data():
    data_nor = []
    data_pne = []
    data_cov = []
    data_covaug = []

    
    for img in tqdm(os.listdir(NOR_DIR)):  

        
        path = os.path.join(NOR_DIR,img)
        img = cv2.imread(path)
        try:
            img = cv2.resize(img, (IMAGE_WIDTH,IMAGE_HEIGHT))
            data_nor.append(img)
                
        except cv2.error as e:
            print('Invalid frame!')
        cv2.waitKey()
            
            
            
    for img in tqdm(os.listdir(PNE_DIR)):
        
        path = os.path.join(PNE_DIR,img)
        img = cv2.imread(path)
        try:
            img = cv2.resize(img, (IMAGE_WIDTH,IMAGE_HEIGHT))         
            data_pne.append(img)
        except cv2.error as e:
            print('Invalid frame!')
        cv2.waitKey()
        
        
    for img in tqdm(os.listdir(COV_DIR)):
        
        path = os.path.join(COV_DIR,img)
        img = cv2.imread(path)
        #Rotate
        rotate=iaa.Affine(rotate=(-15, 15))
        rotated_image=rotate.augment_image(img)
        #Noise
        gaussian_noise=iaa.AdditiveGaussianNoise(10,20)
        noise_image=gaussian_noise.augment_image(img)
        #Cropping image
        crop = iaa.Crop(percent=(0, 0.3)) # crop image
        corp_image=crop.augment_image(img)
        #flipping image horizontally
        flip_hr=iaa.Fliplr(p=1.0)
        flip_hr_image= flip_hr.augment_image(img)
        #Rescale
        scale_im=iaa.Affine(scale={"x": (1.5, 1.0), "y": (1.5, 1.0)})
        scale_image =scale_im.augment_image(img)

        try:
            img = cv2.resize(img, (IMAGE_WIDTH,IMAGE_HEIGHT))
            img = cv2.resize(rotated_image, (IMAGE_WIDTH,IMAGE_HEIGHT))
            img = cv2.resize(noise_image, (IMAGE_WIDTH,IMAGE_HEIGHT))
            img = cv2.resize(corp_image, (IMAGE_WIDTH,IMAGE_HEIGHT))
            img = cv2.resize(flip_hr_image, (IMAGE_WIDTH,IMAGE_HEIGHT))
            img = cv2.resize(scale_image, (IMAGE_WIDTH,IMAGE_HEIGHT))
            data_cov.append(img)
            data_covaug.append(img)
            data_covaug.append(rotated_image)
            data_covaug.append(noise_image)
            data_covaug.append(corp_image)
            data_covaug.append(flip_hr_image)
            data_covaug.append(scale_image)
        except cv2.error as e:
            print('Invalid frame!')
        cv2.waitKey()


    shuffle(data_nor)
    shuffle(data_pne)
    shuffle(data_cov)
    shuffle(data_covaug)
    return data_nor, data_pne, data_cov, data_covaug
        
    

In [4]:
data_nor, data_pne, data_cov, data_covaug = create_data()

100%|████████████████████████████████████████████████████████████████████████████████| 219/219 [00:28<00:00,  7.79it/s]


In [13]:
round(0.2*len(data_nor))

268

# Sort Data

In [14]:
def sort_data(data_nor, data_pne, data_covaug, data_cov, coef_val, coef_test):
    
    #Paths for no data augmentation
    
    NOR_TRAIN_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\train\\NORMAL'
    NOR_TEST_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\test\\NORMAL'
    NOR_VAL_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\val\\NORMAL'
    PNE_TRAIN_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\train\\PNEUMONIA'
    PNE_TEST_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\test\\PNEUMONIA'
    PNE_VAL_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\val\\PNEUMONIA'
    COV_TRAIN_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\train\\COVID-19'
    COV_TEST_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\test\\COVID-19'
    COV_VAL_DIR0 = DATA_PATH + 'Sorted Data (No Aug)\\val\\COVID-19'
    
    #Paths for data augmentation
    
    NOR_TRAIN_DIR = DATA_PATH + 'Sorted Data\\train\\NORMAL'
    NOR_TEST_DIR = DATA_PATH + 'Sorted Data\\test\\NORMAL'
    NOR_VAL_DIR = DATA_PATH + 'Sorted Data\\val\\NORMAL'
    PNE_TRAIN_DIR = DATA_PATH + 'Sorted Data\\train\\PNEUMONIA'
    PNE_TEST_DIR = DATA_PATH + 'Sorted Data\\test\\PNEUMONIA'
    PNE_VAL_DIR = DATA_PATH + 'Sorted Data\\val\\PNEUMONIA'
    COV_TRAIN_DIR = DATA_PATH + 'Sorted Data\\train\\COVID-19'
    COV_TEST_DIR = DATA_PATH + 'Sorted Data\\test\\COVID-19'
    COV_VAL_DIR = DATA_PATH + 'Sorted Data\\val\\COVID-19'
    

    
    #*****************NORMAL*******************
   
    lim_test = round(coef_test*len(data_nor))
    
    train_data_nor = data_nor[0:len(data_nor)-lim_test]
    test_data_nor = data_nor[len(data_nor)-lim_test:len(data_nor)]
    
    lim_val = round(coef_val*len(train_data_nor))
    
    val_data_nor = data_nor[len(train_data_nor)-lim_val:len(train_data_nor)]
    train_data_nor = data_nor[0:len(train_data_nor)-lim_val]
    
    #We copy the images into a new directory so we can use a data generator
    
    
    count=0
    for img in train_data_nor:
        count=count+1
        os.chdir(NOR_TRAIN_DIR) 
        cv2.imwrite('Normal'+str(count) +'.jpg', img) 
        os.chdir(NOR_TRAIN_DIR0)
        cv2.imwrite('Normal'+str(count) +'.jpg', img)
    
    count=0
    for img in test_data_nor: 
        count=count+1
        os.chdir(NOR_TEST_DIR)
        cv2.imwrite('Normal'+str(count) +'.jpg', img) 
        os.chdir(NOR_TEST_DIR0)
        cv2.imwrite('Normal'+str(count) +'.jpg', img) 
     
    count=0
    for img in val_data_nor:
        count=count+1
        os.chdir(NOR_VAL_DIR)
        cv2.imwrite('Normal'+str(count) +'.jpg', img)         
        os.chdir(NOR_VAL_DIR0)
        cv2.imwrite('Normal'+str(count) +'.jpg', img)
        
    #*****************PNEUMONIA*******************
    
    lim_test = round(coef_test*len(data_pne))
    
    train_data_pne = data_pne[0:len(data_pne)-lim_test]
    test_data_pne = data_pne[len(data_pne)-lim_test:len(data_pne)]
    
    lim_val = round(coef_val*len(train_data_pne))
    
    val_data_pne = data_pne[len(train_data_pne)-lim_val:len(train_data_pne)]
    train_data_pne = data_pne[0:len(train_data_pne)-lim_val]
    
    #We copy the images into a new directory so we can use a data generator
    
    
    count=0
    for img in train_data_pne:
        count=count+1
        os.chdir(PNE_TRAIN_DIR) 
        cv2.imwrite('Pneumonia'+str(count) +'.jpg', img) 
        os.chdir(PNE_TRAIN_DIR0) 
        cv2.imwrite('Pneumonia'+str(count) +'.jpg', img)
    
    count=0
    for img in test_data_pne:
        count=count+1
        os.chdir(PNE_TEST_DIR) 
        cv2.imwrite('Pneumonia'+str(count) +'.jpg', img) 
        os.chdir(PNE_TEST_DIR0) 
        cv2.imwrite('Pneumonia'+str(count) +'.jpg', img) 
    
    count=0
    for img in val_data_pne:
        count=count+1
        os.chdir(PNE_VAL_DIR) 
        cv2.imwrite('Pneumonia'+str(count) +'.jpg', img) 
        os.chdir(PNE_VAL_DIR0) 
        cv2.imwrite('Pneumonia'+str(count) +'.jpg', img)

    #*****************COVID-19 With Aug*******************
    
    lim_test = round(coef_test*len(data_covaug))
    
    train_data_cov = data_covaug[0:len(data_covaug)-lim_test]
    test_data_cov = data_covaug[len(data_covaug)-lim_test:len(data_covaug)]
    
    lim_val = round(coef_val*len(train_data_cov))
    
    val_data_cov = data_covaug[len(train_data_cov)-lim_val:len(train_data_cov)]
    train_data_cov = data_covaug[0:len(train_data_cov)-lim_val]
    
    #We copy the images into a new directory so we can use a data generator
    
    os.chdir(COV_TRAIN_DIR) 
    count=0
    for img in train_data_cov:
        count=count+1
        cv2.imwrite('Covid'+str(count) +'.jpg', img) 
        
    os.chdir(COV_TEST_DIR) 
    count=0
    for img in test_data_cov:
        count=count+1
        cv2.imwrite('Covid'+str(count) +'.jpg', img) 

    os.chdir(COV_VAL_DIR) 
    count=0
    for img in val_data_cov:
        count=count+1
        cv2.imwrite('Covid'+str(count) +'.jpg', img)         
        
        
    #*****************COVID-19 With Aug*******************
    
    lim_test = round(coef_test*len(data_cov))
    
    train_data_cov = data_cov[0:len(data_cov)-lim_test]
    test_data_cov = data_cov[len(data_cov)-lim_test:len(data_cov)]
    
    lim_val = round(coef_val*len(train_data_cov))
    
    val_data_cov = data_cov[len(train_data_cov)-lim_val:len(train_data_cov)]
    train_data_cov = data_cov[0:len(train_data_cov)-lim_val]
    
    #We copy the images into a new directory so we can use a data generator
    
    os.chdir(COV_TRAIN_DIR0) 
    count=0
    for img in train_data_cov:
        count=count+1
        cv2.imwrite('Covid'+str(count) +'.jpg', img) 
        
    os.chdir(COV_TEST_DIR0) 
    count=0
    for img in test_data_cov:
        count=count+1
        cv2.imwrite('Covid'+str(count) +'.jpg', img) 

    os.chdir(COV_VAL_DIR0) 
    count=0
    for img in val_data_cov:
        count=count+1
        cv2.imwrite('Covid'+str(count) +'.jpg', img)      
           
        
    return 

In [16]:
coef_test = 0.2
coef_val = 0.2
sort_data(data_nor, data_pne, data_covaug, data_cov, coef_val, coef_test, )

In [37]:
np.round(len(data_nor))

1341

# Data Generator

In [3]:
def data_generator(batch_size):
    train_datagen = ImageDataGenerator(rescale = 1./255)
    val_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen = ImageDataGenerator(rescale = 1./255)

    BATCH_SIZE = batch_size
    IMAGE_SIZE=128
    IMAGE_SIZE_V=(IMAGE_SIZE, IMAGE_SIZE)

    train_dir = DATA_PATH + 'Sorted Data\\train'
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = IMAGE_SIZE_V,
        batch_size = BATCH_SIZE,
        class_mode = 'categorical'
    )

    test_dir = DATA_PATH + 'Sorted Data\\test'
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size = IMAGE_SIZE_V,
        batch_size = BATCH_SIZE,
        class_mode = 'categorical'
    )

    val_dir = DATA_PATH + 'Sorted Data\\val'
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size = IMAGE_SIZE_V,
        batch_size = BATCH_SIZE,
        class_mode = 'categorical'
    )
    
    return train_generator, test_generator, val_generator


In [4]:
batch_size=32

train_generator, test_generator, val_generator = data_generator(batch_size)

Found 2560 images belonging to 3 classes.
Found 800 images belonging to 3 classes.
Found 640 images belonging to 3 classes.
